# 第2节 – 使用ragas进行RAG评估

使用ragas指标评估最小RAG管道：答案相关性、真实性、上下文精确度。


# 场景
此场景在本地评估一个最小化的检索增强生成（RAG）管道。我们将：
- 定义一个小型的合成文档语料库。
- 嵌入文档并实现一个简单的相似性检索器。
- 使用本地模型（Foundry Local / OpenAI兼容）生成基于事实的答案。
- 计算ragas指标（`answer_relevancy`、`faithfulness`、`context_precision`）。
- 支持快速模式（环境变量 `RAG_FAST=1`），仅计算答案相关性以便快速迭代。

使用此笔记本验证您的本地模型和嵌入堆栈是否能够生成基于事实的答案，然后再扩展到更大的语料库。


### 说明：依赖安装
安装所需的库：
- `foundry-local-sdk` 用于本地模型管理。
- `openai` 客户端接口。
- `sentence-transformers` 用于密集嵌入。
- `ragas` + `datasets` 用于评估和指标计算。
- `langchain-openai` 适配器，用于 ragas 的 LLM 接口。

可以安全地重新运行；如果环境已准备好则可跳过。


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### 说明：核心导入和指标
加载核心库和ragas指标。关键部分包括：
- SentenceTransformer 用于生成嵌入。
- `evaluate` 和选定的 ragas 指标。
- `Dataset` 用于构建评估语料库。
这些导入不会触发远程调用（除了可能的嵌入模型缓存加载）。


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### 解释：玩具语料库和问答标准答案
定义了一个小型的内存语料库（`DOCS`）、一组用户问题以及预期的标准答案。这些可以在不需要外部数据获取的情况下快速、确定性地计算指标。在实际场景中，你会采样生产环境中的查询和精心整理的答案。


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### 说明：服务初始化、嵌入和安全补丁
初始化 Foundry Local 管理器，为 `promptTemplate` 应用一个模式漂移安全补丁，解析模型 ID，创建与 OpenAI 兼容的客户端，并为文档语料库预先计算密集嵌入。这为检索和生成设置了可重复使用的状态。


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### 说明：Retriever 函数
定义了一个简单的向量相似性检索器，使用归一化嵌入的点积计算相似性。返回前 k 个文档（默认 k=2）。在生产环境中，可替换为 ANN 索引（如 FAISS、Chroma、Milvus）以提高扩展性和降低延迟。


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### 说明：生成函数
`generate` 构建一个受限的提示（系统指示仅使用上下文）并调用本地模型。低温度（0.1）更倾向于准确提取而非创造性。返回修剪后的答案文本。


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### 解释：备用客户端初始化
确保 `client` 存在，即使之前的初始化单元被跳过或失败——防止在后续评估步骤中出现 NameError。


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### 说明：评估循环与指标
构建评估数据集（必需列：question、answer、contexts、ground_truths、reference），然后迭代选定的ragas指标。

优化：
- FAST_MODE 仅限于回答相关性，用于快速初步测试。
- 每指标循环避免在某个指标失败时进行完整重新计算。

输出一个字典，键为指标，值为分数（如果失败则为NaN）。


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**免责声明**：  
本文档使用AI翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
